In [1]:
# 将MIDI文件转换为用实验报告中的整数序列表示的音乐片段
# 使用了 https://github.com/angelfaraldo/pymidifile/ 这一软件包将音乐对其到八分音符
from mido import Message
from mido import MidiFile
from pymidifile import *
from reformat_midi import *
import numpy as np
import os

In [6]:
def GetTrackName(track):
    """Extract the track name in a track.
    """
    for msg in track:
        if (msg.type == 'track_name'):
            return msg.name
    
    
def FromMIDIToChromosome(filename):
    """Convert a MIDI file into chromosome representation
    Parameters:
        filename: Path of the MIDI file. The file should be single-track and properly quantized.
    
    Return value:
        a list of numbers in the range 0..28, viz. the chromosome representation. 
        (Pitches beyond the range 1..27 are discarded.)
    [Note] Integer representation of pitches in Chromosome:
        0 = rest,
        1 = F3, 2 = #F3, ... , 26 = #F5, 27 = G5
        28 = duration extension
        unit duration = an eighth note
    [Note] Integer representation of pitches as MIDI note number:
        60 = C4
        53 = F3, 54 = #F3, ... , 78 = #F5, 79 = G5
    [Note] Rule of convertion:
        Chromosome number + 52 == MIDI note number
    
    Notes
    """
    
    #mid = MidiFile(filename)
    #print(mid.ticks_per_beat)
    #for i,track in enumerate(mid.tracks):
    #    print(i,GetTrackName(track))
    #x = int(input())
    #for msg in mid.tracks[0]:
    #    print(msg)
    reformatted = reformat_midi(filename, verbose=False, write_to_file=False, override_time_info=True)
    matrix = mid_to_matrix(reformatted)
    quantizer = quantize_matrix(matrix, stepSize=0.5, quantizeOffsets=True, quantizeDurations=True)
    note_list = np.array(sorted(quantizer, key=lambda x: x[0], reverse=False))
    #print(note_list[:,1]) # unit duration = quarter note
    #print(note_list[:,2])
    #print(note_list[:,1]+note_list[:,2])
    #print(2*np.max(note_list[:,1]+note_list[:,2]))
    length = int(2*np.max(note_list[:,1]+note_list[:,2]))
    output = np.zeros(length)
    #for note in note_list:
    for note in sorted(quantizer, key=lambda x: x[0], reverse=False):
        #print(note)
        start = int(2*note[1])
        dur = int(2*note[2])
        pitch = int(note[0] - 52)
        if (pitch in range(1,28)): # if pitch < 1 (F3) or pitch > 27 (G5), it is neglected (replaced with rests)
            for t in range(start, start+dur):
                output[t] = pitch
    output_list = output.tolist()
    while (len(output_list) >= 32):
        cnt = 0
        cnt2 = 0
        for i in range(31, 0, -1):
            if (output_list[i] == 0):
                cnt += 1
            if (output_list[i] == output_list[i-1]):
                output_list[i] = 28
                cnt2 += 1
                
        if (cnt <= 8):
            print(100, end = ' ')
            for i in range(0, 32):
                print(int(output_list[i]), end = ' ')
            print('')
        output_list = output_list[32:]


In [3]:
def FromMIDIFolderToChromosome(foldername):
    for root, dirs, files in os.walk(foldername):
        for name in files:
            FromMIDIToChromosome(os.path.join(root, name))

In [9]:
FromMIDIToChromosome('tchai.mid')

100 24 28 21 28 28 0 28 21 22 24 26 14 21 0 22 28 14 0 14 16 17 21 28 18 28 28 28 19 28 4 14 28 
100 16 17 16 28 19 28 28 12 17 19 12 21 0 24 28 28 12 21 28 28 0 21 28 24 16 28 21 19 28 11 7 21 
100 23 24 16 21 0 19 28 7 21 28 24 0 28 28 24 28 23 21 2 21 14 5 14 28 28 28 12 28 28 0 28 22 
100 0 21 19 14 17 28 16 28 0 14 28 16 14 28 16 0 12 28 28 0 22 0 21 19 14 17 28 16 28 7 0 14 
100 14 16 0 14 16 0 12 28 0 18 28 20 4 18 20 16 28 8 22 28 24 0 22 24 20 28 28 3 22 25 24 22 
100 25 24 20 28 28 3 22 28 24 8 27 28 9 26 25 7 24 28 6 23 18 21 19 6 24 28 0 18 21 19 18 21 
100 19 2 14 28 28 28 28 0 28 14 28 28 16 17 16 28 19 28 12 17 28 19 12 21 0 24 28 28 28 21 28 28 
100 0 21 28 22 24 26 28 21 0 22 28 14 0 14 16 17 21 28 18 28 28 28 19 28 10 14 28 16 17 16 19 28 
100 19 12 17 19 12 21 0 24 28 28 12 28 21 28 28 0 21 28 24 16 28 21 28 19 28 7 21 28 24 16 28 21 
100 21 19 28 28 7 28 28 0 28 24 28 28 28 28 28 28 22 28 28 20 28 28 19 28 28 28 28 18 28 28 28 0 
100 0 28 28 7 12 17 18 24 27 24 19 0

In [10]:
FromMIDIFolderToChromosome('./data/chopin')

100 0 28 28 28 28 28 7 28 16 28 28 17 19 23 24 21 19 28 28 28 14 28 28 16 12 28 28 28 14 16 17 19 
100 21 28 28 28 0 21 28 19 28 28 17 28 0 16 28 14 13 28 14 28 0 6 7 17 16 28 11 12 11 12 28 14 
100 12 28 28 28 11 23 21 19 16 28 28 17 24 26 28 21 28 19 28 28 14 28 28 16 12 28 11 12 14 16 17 19 
100 21 28 28 28 0 21 28 19 21 19 18 28 28 19 21 22 23 28 28 21 18 19 9 0 28 28 14 28 28 19 23 28 
100 18 28 28 28 19 28 0 28 16 28 28 17 19 21 24 22 13 28 28 28 14 28 0 28 18 28 28 19 21 23 26 24 
100 15 28 28 28 16 17 21 19 13 28 28 28 14 15 19 17 28 28 15 14 11 28 28 28 27 28 24 20 8 5 15 6 
100 7 28 28 28 9 11 12 14 16 28 28 17 26 0 28 26 21 19 28 28 14 28 28 15 12 28 28 13 15 19 20 17 
100 15 28 28 28 10 28 28 12 9 28 28 28 28 11 12 13 14 28 7 28 16 28 28 28 0 16 21 26 28 16 28 0 
100 12 28 0 28 28 28 12 28 16 28 28 28 28 28 12 14 16 28 28 28 14 28 12 28 21 28 19 28 12 28 14 16 
100 17 16 0 28 28 12 24 28 28 28 26 24 16 14 12 14 15 16 0 12 24 23 16 12 24 23 11 17 20 23 26 23 
100 17 12 28 8 

100 0 28 28 23 28 28 28 28 24 28 23 14 21 28 28 23 28 28 28 19 28 28 28 28 23 28 14 28 26 28 28 28 
100 26 28 24 28 0 23 28 28 26 24 28 23 21 28 28 28 23 28 28 18 0 19 21 28 19 28 23 14 28 28 26 28 
100 26 28 28 28 24 28 23 28 28 22 28 28 23 25 0 28 4 26 28 28 28 25 28 23 28 28 28 28 18 28 28 21 
100 23 28 21 0 19 28 28 28 28 13 2 0 21 28 28 23 26 27 24 2 23 21 28 19 28 28 21 28 28 28 18 28 
100 0 19 28 21 28 28 23 28 26 24 28 23 21 0 19 7 0 21 28 28 28 28 28 28 14 12 23 28 28 28 28 21 
100 21 14 28 22 28 28 28 12 28 28 28 11 0 11 19 23 28 28 28 28 24 28 23 21 28 28 28 23 28 28 28 19 
100 19 28 28 28 28 23 28 2 28 26 28 28 28 28 28 24 28 23 28 28 26 24 28 23 21 28 28 28 14 23 28 28 
100 19 21 19 23 26 2 0 14 28 9 28 14 28 2 28 24 28 23 28 28 28 22 11 23 25 28 4 28 26 28 28 10 
100 19 28 21 18 12 19 23 28 28 0 23 28 28 28 28 28 28 28 21 28 19 28 28 28 28 28 28 28 11 26 28 28 
100 26 24 28 23 28 28 21 28 28 28 6 21 0 23 21 19 18 0 19 28 13 19 28 9 0 19 28 28 18 0 28 28 
100 23 21 20 21 2

100 0 28 28 28 28 28 28 28 23 28 24 28 21 28 23 28 19 28 23 0 26 28 28 24 23 24 26 24 21 28 23 28 
100 21 28 23 0 26 28 28 24 23 28 22 23 25 28 26 28 23 28 18 28 21 28 16 0 28 19 21 19 23 28 28 21 
100 19 28 28 28 21 28 23 28 26 24 23 21 19 28 21 28 18 28 19 28 21 28 23 28 26 24 23 21 19 28 21 28 
100 21 28 23 28 21 28 22 28 23 28 24 28 21 28 23 28 19 28 23 0 26 28 28 24 23 24 26 24 21 28 23 28 
100 21 23 26 0 28 28 28 24 23 28 22 23 25 28 26 28 23 28 18 28 21 28 16 0 28 19 21 19 23 28 28 21 
100 19 28 26 28 24 28 23 28 21 28 28 28 23 21 19 18 19 28 28 28 28 28 18 28 0 23 20 21 26 21 28 19 
100 19 28 28 28 28 28 18 28 0 28 28 18 19 18 16 14 16 28 28 28 28 28 14 28 0 13 16 20 21 28 28 28 
100 0 21 23 21 18 28 0 28 28 11 14 18 19 28 28 28 18 19 21 28 16 28 23 28 28 28 24 28 28 28 28 28 
100 23 21 19 18 16 28 18 28 15 28 24 28 28 28 28 28 23 21 19 18 16 28 18 28 15 28 23 28 21 28 16 0 
100 0 19 21 19 23 28 28 21 26 28 28 28 25 28 28 28 18 28 24 28 23 28 19 28 16 0 28 28 28 28 24 21 
100 1

100 13 26 28 25 28 6 27 25 23 21 28 20 25 24 23 21 28 28 18 28 28 28 28 28 28 28 28 0 4 13 9 16 
100 4 21 28 18 20 21 26 25 28 28 28 23 28 25 2 23 21 16 28 28 28 21 28 28 23 28 25 28 23 21 23 25 
100 25 28 28 28 0 9 6 13 26 1 25 6 25 1 25 28 26 0 25 20 21 28 28 18 1 0 1 9 13 1 26 0 
100 0 1 9 13 1 0 1 11 26 25 20 21 28 28 28 18 28 28 28 28 28 28 28 28 10 16 22 23 25 23 28 26 
100 25 28 28 28 28 27 24 25 27 20 17 0 28 9 6 0 26 18 21 28 28 13 20 28 18 0 1 9 6 0 26 0 
100 21 28 28 28 20 28 18 0 28 28 28 6 2 12 28 0 18 19 28 28 28 28 11 17 28 18 28 28 28 28 28 28 
100 18 28 28 0 28 25 28 28 28 13 0 24 28 28 12 0 23 25 23 2 20 0 18 16 28 28 0 18 20 28 28 28 
100 21 23 28 28 25 26 18 4 16 0 25 28 28 28 28 28 28 28 28 13 24 28 28 28 0 22 24 22 0 17 19 28 
100 19 0 22 0 24 10 17 18 28 28 6 0 20 28 28 28 21 28 28 4 0 28 25 28 28 28 28 0 24 28 28 3 
100 0 23 25 23 2 20 28 4 16 28 28 18 20 28 0 28 21 0 23 28 28 25 26 18 4 0 16 0 25 28 28 28 
100 0 28 25 28 28 28 28 0 24 28 28 28 28 28 0 28 28 22 

100 19 17 28 28 28 28 28 28 10 28 28 28 22 28 28 21 28 28 28 28 24 28 23 21 16 0 17 28 28 28 16 28 
100 9 28 28 28 0 28 28 28 12 28 16 19 21 22 19 28 24 22 19 17 21 22 19 28 24 28 28 28 28 28 28 28 
100 24 22 28 24 21 28 28 28 9 13 14 16 17 19 16 28 21 19 16 9 17 19 14 28 16 28 28 28 28 28 24 28 
100 24 23 28 24 21 28 28 28 23 28 28 21 24 28 28 23 11 21 28 28 20 23 28 28 16 18 15 13 28 28 28 11 
100 23 28 24 28 26 0 26 28 21 28 24 28 28 28 28 28 28 28 21 28 28 28 26 0 28 26 21 16 28 28 17 26 
100 0 28 28 26 23 16 28 28 17 23 26 23 17 16 28 28 19 24 26 24 19 16 28 28 28 28 28 28 28 28 28 28 
100 0 5 11 9 0 4 12 9 0 4 12 9 0 4 13 9 0 4 13 9 2 5 14 9 2 5 14 9 0 4 14 11 
100 0 2 11 9 0 4 12 9 0 4 12 9 0 4 13 7 0 4 13 7 2 5 14 9 5 2 5 0 28 28 5 2 
100 0 4 12 9 0 2 11 9 0 28 4 12 9 0 2 11 9 0 4 12 9 0 2 11 9 0 4 13 9 4 1 0 
100 16 14 23 28 21 19 28 18 28 14 26 17 26 17 26 28 24 23 28 21 24 28 23 28 21 28 19 18 16 14 23 28 
100 21 19 28 18 28 16 26 28 28 28 28 28 24 23 28 21 23 19 11 16 28 18

100 9 28 14 18 23 0 19 21 19 18 15 11 28 0 11 0 11 0 26 28 28 23 14 16 18 0 19 28 18 28 28 16 
100 14 28 13 28 9 0 18 19 18 16 13 9 28 0 9 0 9 28 14 23 21 0 19 21 20 18 17 15 28 14 18 16 
100 16 9 14 26 23 21 18 24 23 21 18 28 12 28 11 14 11 26 0 28 28 28 26 19 18 0 19 28 18 28 28 16 
100 14 28 28 28 0 28 13 14 16 28 21 23 25 0 25 0 25 28 0 24 19 17 28 28 28 28 21 25 26 28 0 26 
100 26 28 0 25 19 17 28 28 28 28 23 25 26 28 28 28 28 28 28 25 23 25 22 25 23 28 28 22 19 21 19 23 
100 19 28 18 28 13 18 28 19 25 19 23 19 25 21 26 21 19 21 26 20 25 28 28 28 28 19 23 19 25 21 26 21 
100 19 21 26 20 25 21 27 24 27 24 26 24 26 24 19 24 19 24 17 26 28 24 28 28 28 28 26 24 26 24 19 24 
100 22 25 22 26 22 25 24 26 22 25 21 26 17 22 16 22 26 28 25 22 28 28 28 28 28 28 21 4 10 28 28 28 
100 10 28 28 28 28 28 28 9 28 0 18 19 18 16 13 28 9 0 9 0 9 28 14 23 21 0 19 21 20 27 26 24 
100 22 19 14 18 16 9 18 26 23 21 19 21 19 18 15 28 11 0 11 0 11 26 0 28 28 28 28 26 18 0 19 28 
100 0 26 28 21 26 23 19 18 

100 0 28 21 28 18 20 21 26 25 28 28 28 28 28 23 28 25 23 21 17 16 28 28 28 21 28 23 28 25 23 21 23 
100 22 28 25 28 28 23 26 28 25 28 28 28 24 28 26 24 25 27 0 28 28 28 28 28 28 26 21 28 28 28 20 0 
100 20 0 21 0 23 28 28 0 25 26 18 28 16 28 25 28 28 28 28 0 25 28 28 28 28 0 24 28 28 28 28 0 
100 22 24 22 0 17 0 19 28 28 28 28 0 22 24 22 0 17 0 18 28 28 28 28 0 20 28 28 28 28 0 21 28 
100 21 28 28 0 25 28 28 28 28 0 24 28 28 28 28 0 23 25 23 0 20 0 18 0 16 28 28 0 18 0 20 28 
100 20 0 21 0 23 28 28 0 25 26 18 0 16 0 25 28 28 28 28 0 25 28 28 28 28 0 24 28 28 28 28 0 
100 22 24 22 0 17 0 19 28 28 28 28 0 22 24 22 0 17 0 18 28 28 28 28 0 20 28 28 28 28 0 21 28 
100 21 28 28 0 23 28 28 28 28 0 25 28 28 28 28 0 26 0 25 26 16 18 28 0 23 0 16 28 28 0 18 16 
100 26 25 28 0 26 25 23 21 23 28 28 28 28 28 28 28 28 0 24 0 25 28 28 28 28 0 24 28 28 28 28 0 
100 22 24 22 0 17 0 19 28 28 28 28 0 22 24 22 0 17 0 18 28 28 28 28 0 20 28 28 28 28 0 21 28 
100 21 28 28 0 23 28 28 28 28 0 25 28 28 28 28 0